In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor as RFR
import lightgbm as lgb
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import xgboost as xgb

In [2]:
'''

LightGBM


'''

'\n\nLightGBM\n\n\n'

In [3]:
train = pd.read_csv("/home/ishikawa/signate/home_price/train.csv")
test  = pd.read_csv("/home/ishikawa/signate/home_price/test.csv")
#train = train[train['SalePrice'] < 200000]

In [4]:
train['SalePrice'] = np.log(train['SalePrice'])
train = train.drop(columns=['Garage Area'])
test = test.drop(columns=[ 'Garage Area'])

#train['Total_Bsmt'] = train['BsmtFin SF 1'] + train['Bsmt Unf SF']
#test['Total_Bsmt'] = test['BsmtFin SF 1'] + test['Bsmt Unf SF']

#外れ値除去で置換するときはwhereを使う
#train['Total Bsmt SF'] = train['Total Bsmt SF'].where(train['Total Bsmt SF'] > 0, train['Total_Bsmt'])
#test['Total Bsmt SF'] = test['Total Bsmt SF'].where(test['Total Bsmt SF'] > 0, test['Total_Bsmt'])

#train = train.drop(columns=['Bsmt Full Bath','Year Remod/Add'])
#test = test.drop(columns=[ 'Bsmt Full Bath','Year Remod/Add'])

#train = train.drop(columns=['Total_Bsmt'])
#test = test.drop(columns=[ 'Total_Bsmt'])

In [5]:
category = train.columns[train.dtypes == "object"]

In [6]:
category = ['MS Zoning', 'Lot Shape', 'Land Contour', 'Lot Config', 'Neighborhood',
       'Bldg Type', 'House Style', 'Roof Style', 'Exterior 1st',
       'Exterior 2nd', 'Exter Qual', 'Foundation', 'Heating QC', 'Central Air',
       'Electrical', 'Kitchen Qual', 'Paved Drive', 'Sale Type',
       'Sale Condition']

In [7]:
for c in category:
    train[c] = train[c].astype('category')
    test[c]  = test[c].astype('category')

In [8]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 1223)

In [9]:
train_y = pd.DataFrame(data = train['SalePrice'])
del train['SalePrice']
train_x = pd.DataFrame(data = train)
del train

In [10]:
params = {'objective':'regression', 'metric':'rmse', 'seed':2020, 'verbose':0,
         'lambda_l1': 0.0005110547098216984,
         'lambda_l2': 0.006664459019340734,
         'num_leaves': 25,
         'feature_fraction': 0.41600000000000004,
         'bagging_fraction': 1.0,
         'bagging_freq': 0,
         'min_child_samples': 20}

In [11]:
lgb_preds = []
rmses = []
lgb_va_idxes = []
lgb_va_preds = []

for i, (tr_idx,va_idx) in enumerate(kfold.split(train_x)):
    
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    lgb_train = lgb.Dataset(tr_x, tr_y, categorical_feature = category)
    lgb_eval  = lgb.Dataset(va_x, va_y, categorical_feature = category)
    
    lgb_model = lgb.train(params, lgb_train, num_boost_round = 2000, early_stopping_rounds = 100,
                         valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval])
    
    va_pred = lgb_model.predict(va_x,num_iteration = lgb_model.best_iteration)
    
    tmp_rmse = np.sqrt(mean_squared_error(np.exp(va_pred), np.exp(va_y))) #SalePriceのlog変換対応
    
    rmses.append(tmp_rmse)
    print(tmp_rmse)
    
    lgb_va_preds.append(va_pred)
    lgb_va_idxes.append(va_idx)
    
    pred = lgb_model.predict(test, num_iteration = lgb_model.best_iteration)
    lgb_preds.append(pred) #SalePriceのlog変換対応
    

116517	valid's rmse: 0.139255
[60]	train's rmse: 0.116072	valid's rmse: 0.139393
[61]	train's rmse: 0.115484	valid's rmse: 0.139709
[62]	train's rmse: 0.114931	valid's rmse: 0.139772
[63]	train's rmse: 0.114544	valid's rmse: 0.139834
[64]	train's rmse: 0.114055	valid's rmse: 0.139743
[65]	train's rmse: 0.113567	valid's rmse: 0.139907
[66]	train's rmse: 0.112971	valid's rmse: 0.139846
[67]	train's rmse: 0.112378	valid's rmse: 0.140005
[68]	train's rmse: 0.111813	valid's rmse: 0.140018
[69]	train's rmse: 0.111278	valid's rmse: 0.139946
[70]	train's rmse: 0.110915	valid's rmse: 0.140054
[71]	train's rmse: 0.110338	valid's rmse: 0.139765
[72]	train's rmse: 0.109849	valid's rmse: 0.139907
[73]	train's rmse: 0.109375	valid's rmse: 0.140198
[74]	train's rmse: 0.108811	valid's rmse: 0.140624
[75]	train's rmse: 0.108449	valid's rmse: 0.140748
[76]	train's rmse: 0.108055	valid's rmse: 0.14076
[77]	train's rmse: 0.107722	valid's rmse: 0.140807
[78]	train's rmse: 0.107335	valid's rmse: 0.140939
[7

In [12]:
lgb_va_preds

96866936, 11.98104582, 11.91597281, 12.07938988, 11.97350284,
        11.97051834, 11.77611726, 11.83089333, 11.97827451, 12.07228611,
        11.97650079, 11.84520937, 11.89173036, 11.89039169, 11.9242377 ,
        12.03287653, 12.14989634, 11.76306805, 11.806612  , 11.9216752 ,
        12.04036951, 11.74676573, 11.89563368, 11.97121416, 12.01582859,
        11.79023581, 11.93148022, 11.78695091, 11.73185462, 11.87498761,
        11.94421773, 11.98454684, 11.943669  , 11.81632738, 11.7373451 ,
        11.92246373, 12.03684407, 11.87699335, 11.97415147, 11.97237128,
        11.97801743, 11.97790092, 11.88757188, 11.8663472 , 11.96740133,
        11.9654736 , 11.97408216, 11.82902896, 11.78688594, 11.9232482 ,
        12.05631331, 11.77895883, 11.92596153, 11.89011739, 11.91208579,
        11.87917825, 11.85043404, 11.61737306, 11.8065988 , 11.9933722 ,
        11.86728664, 11.97205951, 11.93158912, 12.06548531, 11.94404612,
        11.81813802, 11.69326402, 11.98612792, 12.03460743, 11

In [13]:
va_idxes_lgb = np.concatenate(lgb_va_idxes)
va_pred_lgb = np.concatenate(lgb_va_preds,axis = 0)
#order = np.argsort(va_idxes_lgb)
va_pred_lgb = pd.DataFrame(data = va_pred_lgb)
order       = pd.DataFrame(data = va_idxes_lgb)
pred_train_lgb = pd.concat([order,va_pred_lgb],axis = 1)
pred_train_lgb.columns = ["id","Pred_LGBM"]

preds_test_lgb = np.mean(lgb_preds, axis = 0)

In [14]:
pred_train_lgb.head()

,id,Pred_LGBM
0,1,11.920807
1,7,11.825732
2,10,12.134273
3,16,11.992688
4,23,12.134359


In [15]:
'''

RandomForest


'''

'\n\nRandomForest\n\n\n'

In [16]:
train = pd.read_csv("/home/ishikawa/signate/home_price/train.csv")
test  = pd.read_csv("/home/ishikawa/signate/home_price/test.csv")
#train = train[train['SalePrice'] < 200000]

In [17]:
category = train.columns[train.dtypes == "object"]
all_x = pd.concat([train, test])
all_x_one = pd.get_dummies(all_x, columns = category)

In [18]:
train_one = all_x_one.iloc[:train.shape[0], :].reset_index(drop = True)
test_one  = all_x_one.iloc[train.shape[0]:, :].reset_index(drop = True)

In [19]:
train = pd.DataFrame(data = train_one)
test = pd.DataFrame(data = test_one)

In [20]:
del train_one
del test_one

In [21]:
train['SalePrice'] = np.log(train['SalePrice'])

train = train.drop(columns=['Garage Area'])
test = test.drop(columns=[ 'Garage Area'])

#train['Total_Bsmt'] = train['BsmtFin SF 1'] + train['Bsmt Unf SF']
#test['Total_Bsmt'] = test['BsmtFin SF 1'] + test['Bsmt Unf SF']

#train['Total Bsmt SF'] = train['Total Bsmt SF'].where(train['Total Bsmt SF'] > 0, train['Total_Bsmt'])
#test['Total Bsmt SF'] = test['Total Bsmt SF'].where(test['Total Bsmt SF'] > 0, test['Total_Bsmt'])

#train = train.drop(columns=['Bsmt Full Bath','Year Remod/Add'])
#test = test.drop(columns=[ 'Bsmt Full Bath','Year Remod/Add'])

#train = train.drop(columns=['Total_Bsmt'])
#test = test.drop(columns=[ 'Total_Bsmt'])

In [22]:
train_y = pd.DataFrame(train['SalePrice'])
del train['SalePrice']
train_x = pd.DataFrame(data = train)
del train

In [23]:
del test['SalePrice']

In [24]:
RF_model = RFR(n_estimators = 1500, random_state = 1223, max_depth = 70 )
kFold = KFold(n_splits = 10, shuffle = True, random_state = 2020)
preds_RF = []
rmses = []
va_idxes_RF = []
va_preds_RF = []

for tr_idx, va_idx in kFold.split(train_x):
    
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    RF_model.fit(tr_x, tr_y)
    
    va_pred = RF_model.predict(va_x)
    
    tmp_rmse = np.sqrt(mean_squared_error(np.exp(va_pred), np.exp(va_y)))
    rmses.append(tmp_rmse)
    
    va_preds_RF.append(va_pred)
    va_idxes_RF.append(va_idx)
    
    pred = RF_model.predict(test)
    preds_RF.append(pred)
    

In [25]:
va_idxes_RF = np.concatenate(va_idxes_RF)
va_pred_RF = np.concatenate(va_preds_RF,axis = 0)
#order = np.argsort(va_idxes_RF)
va_pred_RF = pd.DataFrame(data = va_pred_RF)
order       = pd.DataFrame(data = va_idxes_RF)
pred_train_RF = pd.concat([order,va_pred_RF],axis = 1)

pred_train_RF.columns = ["id","Pred_RF"]

preds_test_RF = np.mean(preds_RF, axis = 0)

pred_train_RF.head()

,id,Pred_RF
0,10,12.095845
1,13,11.786684
2,14,12.085356
3,31,11.915181
4,37,11.965273


In [26]:
'''

Ridge Liner regression


'''

'\n\nRidge Liner regression\n\n\n'

In [27]:
train = pd.read_csv("/home/ishikawa/signate/home_price/train.csv")
test  = pd.read_csv("/home/ishikawa/signate/home_price/test.csv")

#train = train[train['SalePrice'] < 200000]

category = train.columns[train.dtypes == "object"]
all_x = pd.concat([train, test])
all_x_one = pd.get_dummies(all_x, columns = category)

train_one = all_x_one.iloc[:train.shape[0], :].reset_index(drop = True)
test_one  = all_x_one.iloc[train.shape[0]:, :].reset_index(drop = True)

train = pd.DataFrame(data = train_one)
test = pd.DataFrame(data = test_one)

del train_one
del test_one

train['SalePrice'] = np.log(train['SalePrice'])

train = train.drop(columns=['Garage Area'])
test = test.drop(columns=[ 'Garage Area'])
'''
train['Total_Bsmt'] = train['BsmtFin SF 1'] + train['Bsmt Unf SF']
test['Total_Bsmt'] = test['BsmtFin SF 1'] + test['Bsmt Unf SF']

train['Total Bsmt SF'] = train['Total Bsmt SF'].where(train['Total Bsmt SF'] > 0, train['Total_Bsmt'])
test['Total Bsmt SF'] = test['Total Bsmt SF'].where(test['Total Bsmt SF'] > 0, test['Total_Bsmt'])

train = train.drop(columns=['Bsmt Full Bath','Year Remod/Add'])
test = test.drop(columns=[ 'Bsmt Full Bath','Year Remod/Add'])

train = train.drop(columns=['Total_Bsmt'])
test = test.drop(columns=[ 'Total_Bsmt'])
'''
train_y = pd.DataFrame(train['SalePrice'])
del train['SalePrice']
train_x = pd.DataFrame(data = train)
del train

del test['SalePrice']

In [28]:
kFold = KFold(n_splits = 10, shuffle = True, random_state = 2020)
preds_LR = []
rmses = []
va_idxes_LR = []
va_preds_LR = []

ridge_model = linear_model.Ridge(alpha = 0.01)

for tr_idx, va_idx in kFold.split(train_x):
    
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    ridge_model.fit(tr_x,tr_y)
    
    va_pred = ridge_model.predict(va_x)
    va_preds_LR.append(va_pred)
    
    va_idxes_LR.append(va_idx)
    
    tmp_rmse = np.sqrt(mean_squared_error(np.exp(va_pred), np.exp(va_y)))
    rmses.append(tmp_rmse)
    
    pred = ridge_model.predict(test)
    preds_LR.append(pred)
    

In [29]:
va_idxes_LR = np.concatenate(va_idxes_LR)
va_pred_LR = np.concatenate(va_preds_LR,axis = 0)
#order = np.argsort(va_idxes_LR)
va_pred_LR = pd.DataFrame(data = va_pred_LR)
order       = pd.DataFrame(data = va_idxes_LR)
pred_train_LR = pd.concat([order,va_pred_LR],axis = 1)
pred_train_LR.columns = ["id","Pred_LR"]

preds_test_LR = np.mean(preds_LR, axis = 0)

pred_train_LR.head()

,id,Pred_LR
0,10,11.930557
1,13,11.768501
2,14,12.025407
3,31,12.040083
4,37,11.899383


In [30]:
'''

TabNet


'''

'\n\nTabNet\n\n\n'

In [31]:
train = pd.read_csv("/home/ishikawa/signate/home_price/train.csv")
test  = pd.read_csv("/home/ishikawa/signate/home_price/test.csv")

#train = train[train['SalePrice'] < 200000]

category = train.columns[train.dtypes == "object"]
all_x = pd.concat([train, test])
all_x_one = pd.get_dummies(all_x, columns = category)

train_one = all_x_one.iloc[:train.shape[0], :].reset_index(drop = True)
test_one  = all_x_one.iloc[train.shape[0]:, :].reset_index(drop = True)

train = pd.DataFrame(data = train_one)
test = pd.DataFrame(data = test_one)

del train_one
del test_one

train['SalePrice'] = np.log(train['SalePrice'])

train = train.drop(columns=['Garage Area'])
test = test.drop(columns=[ 'Garage Area'])

#train['Total_Bsmt'] = train['BsmtFin SF 1'] + train['Bsmt Unf SF']
#test['Total_Bsmt'] = test['BsmtFin SF 1'] + test['Bsmt Unf SF']

#train['Total Bsmt SF'] = train['Total Bsmt SF'].where(train['Total Bsmt SF'] > 0, train['Total_Bsmt'])
#test['Total Bsmt SF'] = test['Total Bsmt SF'].where(test['Total Bsmt SF'] > 0, test['Total_Bsmt'])

#train = train.drop(columns=['Bsmt Full Bath','Year Remod/Add'])
#test = test.drop(columns=[ 'Bsmt Full Bath','Year Remod/Add'])

#train = train.drop(columns=['Total_Bsmt'])
#test = test.drop(columns=[ 'Total_Bsmt'])

train_y = pd.DataFrame(train['SalePrice'])
del train['SalePrice']
train_x = pd.DataFrame(data = train)
del train

del test['SalePrice']

In [32]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 1223)

preds_TN = []
rmses = []
va_idxes_TN = []
va_preds_TN = []


for tr_idx,va_idx in kfold.split(train_x):
    
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    tabnet_params = dict(n_d=50, n_a=50, n_steps=3, gamma=1.3,
                         n_independent=2, n_shared=2,
                         seed=1223, lambda_sparse=1e-3, 
                         optimizer_fn=torch.optim.Adam, 
                         optimizer_params=dict(lr=2e-2),
                         mask_type="entmax",
                         scheduler_params=dict(mode="min",
                                               patience=5,
                                               min_lr=1e-5,
                                               factor=0.9,),
                         scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                         verbose=10
                         )

    pretrainer = TabNetPretrainer(**tabnet_params)

    pretrainer.fit(
                    X_train=tr_x.values,
                    eval_set=[va_x.values],
                    max_epochs=200,
                    patience=20, batch_size=200, virtual_batch_size=100,
                    num_workers=0, drop_last=True,pretraining_ratio=0.8)
    
    tabnet_params = dict(n_d=50, n_a=50, n_steps=3, gamma=1.3,
                         n_independent=2, n_shared=2,
                         seed=1223, lambda_sparse=1e-3,
                         optimizer_fn=torch.optim.Adam,
                         optimizer_params=dict(lr=2e-2,
                                               weight_decay=1e-5
                                              ),
                         mask_type="entmax",
                         scheduler_params=dict(max_lr=0.05,
                                               steps_per_epoch=int(tr_x.shape[0] /100),
                                               epochs=200,
                                               is_batch_level=True
                                              ),
                         scheduler_fn=torch.optim.lr_scheduler.OneCycleLR,
                         verbose=10,
                         #cat_idxs=cat_idxs, # comment out when Unsupervised
                         #cat_dims=cat_dims, # comment out when Unsupervised
                         #cat_emb_dim=1 # comment out when Unsupervised
                        )
    model = TabNetRegressor(**tabnet_params)

    model.fit(X_train=tr_x.values,
                  y_train=tr_y.values,
                  eval_set=[(va_x.values, va_y.values)],
                  eval_name = ["valid"],
                  eval_metric = ["rmse"],
                  max_epochs=1000,
                  patience=20, batch_size=200, virtual_batch_size=100,
                  num_workers=0, drop_last=False,
                  from_unsupervised=pretrainer # comment out when Unsupervised
                 )
    
    va_pred = model.predict(va_x.values)
    va_preds_TN.append(va_pred)
    
    va_idxes_TN.append(va_idx)
    
    pred = model.predict(test.values)
    preds_TN.append(pred)
    
    tmp_rmse = np.sqrt(mean_squared_error(va_pred, va_y))
    rmses.append(tmp_rmse)
    print(tmp_rmse)
    

Device used : cuda
epoch 0  | loss: 291705467.69231| val_0_unsup_loss: 127520520.0|  0:00:00s
epoch 10 | loss: 97570.64123| val_0_unsup_loss: 66774.16406|  0:00:04s
epoch 20 | loss: 625781.71184| val_0_unsup_loss: 100130.44531|  0:00:08s
epoch 30 | loss: 99660.9357| val_0_unsup_loss: 40729.91016|  0:00:11s
epoch 40 | loss: 76886.40835| val_0_unsup_loss: 40388.91016|  0:00:15s
epoch 50 | loss: 46859.76337| val_0_unsup_loss: 133626.39062|  0:00:18s
epoch 60 | loss: 39527.21785| val_0_unsup_loss: 32096.03516|  0:00:22s
epoch 70 | loss: 38879.08203| val_0_unsup_loss: 33220.35938|  0:00:25s
epoch 80 | loss: 35451.77524| val_0_unsup_loss: 31514.89648|  0:00:29s
epoch 90 | loss: 35228.23227| val_0_unsup_loss: 30432.14453|  0:00:33s

Early stopping occured at epoch 91 with best_epoch = 71 and best_val_0_unsup_loss = 29641.92969
Best weights from best epoch are automatically used!
Device used : cuda
Loading weights from unsupervised pretraining
epoch 0  | loss: 143.66849| valid_rmse: 11.9744 | 

In [33]:
va_idxes_TN = np.concatenate(va_idxes_TN)
va_pred_TN = np.concatenate(va_preds_TN,axis = 0)
va_pred_TN = pd.DataFrame(data = va_pred_TN)
order       = pd.DataFrame(data = va_idxes_TN)
pred_train_TN = pd.concat([order,va_pred_TN],axis = 1)
pred_train_TN.columns = ["id","Pred_TN"]

preds_test_TN = np.mean(preds_TN, axis = 0)

pred_train_TN.head()

,id,Pred_TN
0,1,11.929918
1,7,11.796266
2,10,11.932304
3,16,11.927071
4,23,11.961583


In [34]:
pred_train_TN.tail()

,id,Pred_TN
2995,2899,11.823657
2996,2901,11.890195
2997,2979,11.744009
2998,2982,11.810146
2999,2995,11.839073


In [35]:
'''XGboost'''

'XGboost'

In [36]:
train = pd.read_csv("/home/ishikawa/signate/home_price/train.csv")
test  = pd.read_csv("/home/ishikawa/signate/home_price/test.csv")

#train = train[train['SalePrice'] < 200000]

category = train.columns[train.dtypes == "object"]
all_x = pd.concat([train, test])
all_x_one = pd.get_dummies(all_x, columns = category)

train_one = all_x_one.iloc[:train.shape[0], :].reset_index(drop = True)
test_one  = all_x_one.iloc[train.shape[0]:, :].reset_index(drop = True)

train = pd.DataFrame(data = train_one)
test = pd.DataFrame(data = test_one)

del train_one
del test_one

train['SalePrice'] = np.log(train['SalePrice'])

train = train.drop(columns=['Garage Area'])
test = test.drop(columns=[ 'Garage Area'])

#train['Total_Bsmt'] = train['BsmtFin SF 1'] + train['Bsmt Unf SF']
#test['Total_Bsmt'] = test['BsmtFin SF 1'] + test['Bsmt Unf SF']

#train['Total Bsmt SF'] = train['Total Bsmt SF'].where(train['Total Bsmt SF'] > 0, train['Total_Bsmt'])
#test['Total Bsmt SF'] = test['Total Bsmt SF'].where(test['Total Bsmt SF'] > 0, test['Total_Bsmt'])

#train = train.drop(columns=['Bsmt Full Bath','Year Remod/Add'])
#test = test.drop(columns=[ 'Bsmt Full Bath','Year Remod/Add'])

#train = train.drop(columns=['Total_Bsmt'])
#test = test.drop(columns=[ 'Total_Bsmt'])

train_y = pd.DataFrame(train['SalePrice'])
del train['SalePrice']
train_x = pd.DataFrame(data = train)
del train

del test['SalePrice']

In [37]:
KFold = KFold(n_splits = 10, shuffle = True, random_state = 1223)
xgb_params = {"eta":0.01,
               "seed":2020,
                "max_depth":20,
                'objective':'reg:squarederror',
                'eval_metric':'rmse',
                'tree_method':'gpu_hist' ,
                'lambda':3
                }

In [38]:
num_round = 2000

preds_XG = []
rmses = []
va_idxes_XG = []
va_preds_XG = []

for tr_idx,va_idx in KFold.split(train_x):
    
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    dtrain = xgb.DMatrix(tr_x, label = tr_y)
    dvalid  = xgb.DMatrix(va_x, label = va_y)

    dtest  = xgb.DMatrix(test)

    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    
    xgb_model = xgb.train(xgb_params,dtrain, num_round,evals = watchlist)

    va_pred = xgb_model.predict(dvalid)

    va_preds_XG.append(va_pred)
    
    va_idxes_XG.append(va_idx)

    tmp_rmse = np.sqrt(mean_squared_error(va_pred, va_y))
    rmses.append(tmp_rmse)
    print(tmp_rmse)

    pred = xgb_model.predict(dtest)
    preds_XG.append(pred) 

393
[1546]	train-rmse:0.00234	eval-rmse:0.16393
[1547]	train-rmse:0.00234	eval-rmse:0.16393
[1548]	train-rmse:0.00233	eval-rmse:0.16393
[1549]	train-rmse:0.00233	eval-rmse:0.16393
[1550]	train-rmse:0.00232	eval-rmse:0.16393
[1551]	train-rmse:0.00231	eval-rmse:0.16393
[1552]	train-rmse:0.00231	eval-rmse:0.16393
[1553]	train-rmse:0.00230	eval-rmse:0.16393
[1554]	train-rmse:0.00230	eval-rmse:0.16393
[1555]	train-rmse:0.00229	eval-rmse:0.16393
[1556]	train-rmse:0.00229	eval-rmse:0.16393
[1557]	train-rmse:0.00228	eval-rmse:0.16393
[1558]	train-rmse:0.00227	eval-rmse:0.16393
[1559]	train-rmse:0.00227	eval-rmse:0.16393
[1560]	train-rmse:0.00226	eval-rmse:0.16393
[1561]	train-rmse:0.00226	eval-rmse:0.16393
[1562]	train-rmse:0.00225	eval-rmse:0.16393
[1563]	train-rmse:0.00224	eval-rmse:0.16393
[1564]	train-rmse:0.00224	eval-rmse:0.16393
[1565]	train-rmse:0.00223	eval-rmse:0.16393
[1566]	train-rmse:0.00223	eval-rmse:0.16394
[1567]	train-rmse:0.00222	eval-rmse:0.16394
[1568]	train-rmse:0.00221	ev

In [39]:
va_idxes_XG = np.concatenate(va_idxes_XG)
va_pred_XG = np.concatenate(va_preds_XG,axis = 0)
va_pred_XG = pd.DataFrame(data = va_pred_XG)
order       = pd.DataFrame(data = va_idxes_XG)
pred_train_XG = pd.concat([order,va_pred_XG],axis = 1)
pred_train_XG.columns = ["id","Pred_XG"]

preds_test_XG = np.mean(preds_XG, axis = 0)

pred_train_XG.head()

,id,Pred_XG
0,1,11.847257
1,7,11.829121
2,10,12.130093
3,16,12.020136
4,23,12.012076


In [40]:
pred_train_lgb.shape

(3000, 2)

In [41]:
pred_train_RF.shape

(3000, 2)

In [42]:
pred_train_LR.shape

(3000, 2)

In [43]:
pred_train_lgb.head()

,id,Pred_LGBM
0,1,11.920807
1,7,11.825732
2,10,12.134273
3,16,11.992688
4,23,12.134359


In [44]:
pred_train_lgb = pred_train_lgb.sort_values('id')
pred_train_lgb.head()

,id,Pred_LGBM
900,0,11.965755
0,1,11.920807
1200,2,11.818484
2400,3,11.876481
1500,4,12.091932


In [45]:
pred_train_RF = pred_train_RF.sort_values('id')
pred_train_LR = pred_train_LR.sort_values('id')
pred_train_TN = pred_train_TN.sort_values('id')
pred_train_XG = pred_train_XG.sort_values('id')

In [46]:
train_pred = pd.merge(pred_train_lgb, pred_train_RF, how = 'left', on = 'id')
train_pred = pd.merge(train_pred,pred_train_LR, how = 'left', on = 'id')
train_pred = pd.merge(train_pred,pred_train_TN, how = 'left', on = 'id')
train_pred = pd.merge(train_pred,pred_train_XG, how = 'left', on = 'id')
train_pred.head()
#test_pred = pd.DataFrame({'pred_LGBM':pred_test_lgb,'pred_RF':pred_test_RF,'pred_LR':pred_test_LR})

,id,Pred_LGBM,Pred_RF,Pred_LR,Pred_TN,Pred_XG
0,0,11.965755,11.971192,11.982976,11.909129,11.970339
1,1,11.920807,11.899078,11.990569,11.929918,11.847257
2,2,11.818484,11.788391,12.014525,11.978004,11.794640
3,3,11.876481,11.975346,11.826062,11.734773,11.877540
4,4,12.091932,11.918731,11.912285,11.945896,12.054052


In [47]:
del train_pred['id']

In [48]:
'''

2層目
ridge回帰


'''

'\n\n2層目\nridge回帰\n\n\n'

In [49]:
#KFold = KFold(n_splits = 5, shuffle = True, random_state = 1223)
#KFold = KFold(n_splits = 5, shuffle = True, random_state = 1223)

In [50]:
preds_test_lgb = pd.DataFrame(data=preds_test_lgb, columns = ['Preds_lgb'])
preds_test_RF  = pd.DataFrame(data=preds_test_RF, columns = ['Preds_RF'])
preds_test_LR =  pd.DataFrame(data=preds_test_LR, columns = ['Preds_LR'])
preds_test_TN =  pd.DataFrame(data=preds_test_TN, columns = ['Preds_TN'])
preds_test_XG =  pd.DataFrame(data=preds_test_XG, columns = ['Preds_XG'])

test_pred = pd.concat([preds_test_lgb,preds_test_RF],axis = 1)
test_pred = pd.concat([test_pred,preds_test_LR], axis = 1)
test_pred = pd.concat([test_pred,preds_test_TN], axis = 1)
test_pred = pd.concat([test_pred,preds_test_XG], axis = 1)
test_pred.head()

,Preds_lgb,Preds_RF,Preds_LR,Preds_TN,Preds_XG
0,11.892250,11.914471,11.852434,11.995636,11.874422
1,11.956680,11.971073,11.928764,11.903925,11.904577
2,11.906136,11.926738,11.811534,11.918651,11.817995
3,11.873929,11.865620,11.901240,11.905617,11.826089
4,11.972198,11.970550,11.921383,11.924900,11.971805


In [51]:
#KFold = KFold(n_splits = 5, shuffle = True, random_state = 1223)
preds_2nd_LR = []
rmses_2nd = []
va_idxes_2nd_LR = []
va_preds_2nd_LR = []

ridge_2nd_model = linear_model.Ridge(alpha = 0.01)

for tr_idx, va_idx in KFold.split(train_pred):
    
    tr_x, va_x = train_pred.iloc[tr_idx], train_pred.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    ridge_2nd_model.fit(tr_x,tr_y)
    
    va_pred = ridge_2nd_model.predict(va_x)
    
    tmp_rmse = np.sqrt(mean_squared_error(np.exp(va_pred), np.exp(va_y)))
    rmses_2nd.append(tmp_rmse)
    
    pred = ridge_2nd_model.predict(test_pred)
    preds_2nd_LR.append(np.exp(pred))
    

In [52]:
rmse_mean =  sum(rmses_2nd)/len(rmses_2nd)
print(rmse_mean)

26999.244527934312


In [53]:
preds_2nd_mean = np.mean(preds_2nd_LR, axis = 0)

In [54]:
pred_sub = pd.DataFrame(data = preds_2nd_mean)
pred_sub.head()

,0
0,147965.817494
1,155305.110353
2,148005.446212
3,143005.013618
4,158071.132172


In [55]:
test_index = pd.DataFrame(data = test['index'])

In [56]:
sub_data = pd.concat([test_index,pred_sub],axis = 1)
sub_data.head()

,index,0
0,398,147965.817494
1,3833,155305.110353
2,4836,148005.446212
3,4572,143005.013618
4,636,158071.132172


In [58]:
sub_data.to_csv('submission.csv',header = False, index = False)